In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
import torchvision.models as models

import copy

In [4]:
def content_loss(g, c):
    return F.mse_loss(g, c)

def style_loss(g, s):
    return F.mse_loss(g, s) / (g.shape[0] ** 2)

def gram_matrix(x):
    _, B, H, W = x.size()
    x = x.view(B, H * W)
    return torch.matmul(x, x.t())

In [6]:
# cnn = models.vgg19(pretrained=True).features.to(device).eval()

In [ ]:
# Load VGG19 Skeleton
vgg = models.vgg19(pretrained=True)

# Change Pooling Layer
def pool_(model, pool='avg'):
    if (pool=='avg'):
        ct=0
        for layer in model.children():
            if isinstance(layer, nn.MaxPool2d):
                model[ct] = nn.AvgPool2d(kernel_size=2, stride=2, padding=0, ceil_mode=False)
            ct+=1
    elif (pool=='max'):
        ct=0
        for layer in model.children():
            if isinstance(layer, nn.AvgPool2d):
                model[ct] = nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
            ct+=1
            
    return model

#vgg.features = pool_(vgg.features, POOL)

# Extract only the 'features' network, 
# 'classifier' network is not needed
model = copy.deepcopy(vgg.features)
model.to(device)

# Turn-off unnecessary gradient tracking
for param in model.parameters():
    param.requires_grad = False